In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

"""import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [27]:
img=cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask/1022.png")

In [108]:
plt.imshow(img)

In [29]:
img.shape
#cv2.resize(img,(80,80)) #all image must be in same shape# this can be done by imagedatagenerator by giving target size

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
data_gen=ImageDataGenerator(rescale=1 / 255.0,
                   rotation_range=20,
                   zoom_range=0.05,
                   width_shift_range=0.05,
                   height_shift_range=0.05,
                   shear_range=0.05,
                   horizontal_flip=True,
                   fill_mode='reflect',
                   brightness_range=(0.1,0.1)
                   validation_split=0.20)

In [93]:
train_data=data_gen.flow_from_directory("../input/face-mask-12k-images-dataset/Face Mask Dataset/Train",target_size=(80,80),batch_size=32, class_mode='categorical')
#class mode is binary bcz only 2 classes with mask and without mask

In [94]:
train_data.class_indices

In [95]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [96]:
model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(80,80,3)))
model.add(MaxPool2D())
model.add(Conv2D(32, (3,3)))
model.add(MaxPool2D())
model.add(Conv2D(64, (3,3)))
model.add(MaxPool2D())
model.add(Conv2D(128, (3,3)))
model.add(MaxPool2D())
#model.add(Conv2D(256, (3,3)))
#model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [97]:
model.summary() #(None, 78, 78, 16)=(no.of input,imageshape*imageshape,no.of layers)

In [98]:
model.fit(train_data,epochs=2, batch_size=32)

In [100]:
model.save("Face mask fitting.h5")

In [101]:
import os

In [105]:
os.listdir("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test")

# **Predicting single image**

In [178]:
img1163=cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/1163.png")  #its a numpy array(we cant d any operation unless it is converted into array)
plt.imshow(img1163)

In [179]:
img1163.shape

In [180]:
img1163=cv2.resize(img1163,(80,80))/255
print(img1163.shape)
plt.imshow(img1163)

In [181]:
model.predict_on_batch(img1163.reshape(1,80,80,3))    #(1,80,80,3)=(1==no. of inputs,80,80==(shape of image),3==rgb)

In [182]:
model.predict_on_batch(img1163.reshape(1,80,80,3)).argmax()

# **Prediction on whole test dataset**

In [195]:
files=os.listdir("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask")
#for file in files:
print(len(files))
#cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/Augmented_735_9235977.png")

In [191]:
train_data.class_indices

In [198]:
withmask=0
withoutmask=0

for path in files:
    #print(type(path))
    file=cv2.imread("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/"+path)
    #print(file.shape)
    file=cv2.resize(file,(80,80))/255
    #print(file.shape)
    file=file.reshape(1,80,80,3)
    #print(file.shape)
    prediction=model.predict(file).argmax()
    if prediction==0:
        withmask=withmask+1         #{'WithMask': 0, 'WithoutMask': 1}
    else:
        withoutmask=withoutmask+1
print(f"withmask:{withmask},withoutmask:{withoutmask}")

In [196]:
309+174

In [ ]:
# 309 withmask
# 174 withoutmask
#309 correct predictions

In [202]:
# we construct in general function(for test as well as validation)
def predict_mask(mainpath):                       #give mainpath in colon and mainpath should be withmask and withoutmask path of both test and validation
    files=os.listdir(mainpath)
    withmask=0
    withoutmask=0
    for path in files:
        file=cv2.imread(mainpath+"/"+path)
        file=cv2.resize(file,(80,80))/255
        file=file.reshape(1,80,80,3)
        prediction=model.predict(file).argmax()
        if prediction==0:
            withmask=withmask+1
        else:
            withoutmask=withoutmask+1
    print(f"withmask:{withmask},   withoutmask:{withoutmask}")

# **pred on testing data**

In [203]:
predict_mask("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask")   #309 correct pred

In [204]:
predict_mask("../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithoutMask")   # 480 correct pred

# **pred on validation data**

In [206]:
predict_mask("../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask")  #264 correct

In [207]:
predict_mask("../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithoutMask")  #379 correct

In [208]:
model.save("whole pred completed Face mask.h5")